## In what ways has social media contributed to the growth and visibility of the Fridays for Future climate movement?


In [ ]:
from googleapiclient.discovery import build
from collections import OrderedDict
api_key = ""
youtube = build("youtube", "v3", developerKey= api_key)

# Setting parameters for video search
all_videos = []
next_page_token = None
max_pages = 3
current_page = 0

while current_page < max_pages:
    request = youtube.search().list(
        q="Fridays for Future",
        maxResults=50,
        part="snippet",
        order="relevance",
        regionCode="DE",
        type="video",
        publishedAfter="2018-01-01T00:00:00Z",
        publishedBefore="2021-01-01T00:00:00Z",
        pageToken=next_page_token
    )
    response = request.execute()
    all_videos.extend(response["items"])

    next_page_token = response.get("nextPageToken")
    current_page += 1
    
#List of video ids
ids = []
for item in all_videos:
    ids.append(item["id"]["videoId"])

#handle duplicates 
unique_ids= list(set(ids))
print(unique_ids)

titles = []
for item in all_videos:
    titles.append(item["snippet"]["title"])
    

#getting comments for each video
all_comments = [] 
for videoid in unique_ids:
    video_comments = []
    try: 
      request = youtube.commentThreads().list(
        part="snippet",
        videoId= videoid,
        maxResults= 10,
        testFormat="plainText",
        order= "relevance"        
    )
      response = request.execute()
      for c in response.get("items", []):
         video_comments.append(c["snippet"]["topLevelComment"]["snippet"]["textDisplay"])
          
    #Comments disabled
    except Exception as e:
        continue
    all_comments.append(video_comments)

descriptions =[]
for video_id in unique_ids:
    try:
        request = youtube.videos().list(
            part="snippet",
            id=video_id
        )
        response = request.execute()
        for item in response.get("items", []):
            description = item["snippet"].get("description", "")
            descriptions.append(description)

    except Exception as e:
         continue
        
final_data = all_comments + titles + descriptions
print(final_data)



    

['ZuMlFqXelcw', 'l6I2ZUL_k7s', 'rMO7dwlHle8', 'JIPQy_7RYWc', 'BNIwu3eYMg8', '8l-aYsyDgeA', 'hVJCUgL2kKI', 'UMNRwjOspfs', '6UNt-JYSfyo', 'Rb-jAqMhem8', 'E73ag2Fvi3o', '5N8BJ6aYtUg', 'P4Ew7e0MGmo', 'coloDC2r6c8', 'jhv_K9yh-HM', 'OAnhzSPbCxA', 'FkF8jB7ob8c', 'IB5RyvdRAcU', 'izheVHvRII0', '43nxknUKiIg', 'PXYwuXwA4zk', 'DUNA4EqY1zg', 'Z1QBloCU2ws', '4jOp2scnxqo', 'jYFAvYqM328', 'ygVxFcD80IM', 'HVDdsni6GWY', 's0CSk4vSI7A', 'xQvpTl9ESwU', 'CFUNBCH2cRk', 'amvaF4ZsATk', 'cSOo__j0JIA', 'dlFt_fnetIo', 'oXXnjiXvM4U', 'ihKOADLb7ZY', '9OS5YtOLmq4', 'hL5lYDcAItg', 'vM7ZOZTOBjU', 'GKfxPgyxvls', 'uhYwVxypC8A', 'VAnUkKAwKKg', 'ETjsMaC3364', 'nopHNQfxJ8I', 'scJTCK5ZADY', 'Z9Bhx1Zcijk', '3WYGZyLNmN4', 'soTv7jUw7TI', 'x5OKtT7hjfc', 'KpVviTm9a28', 'p8ynEp-XQZ0', '4zxM7PKPS5c', 'HIZqT1prVoQ', 'ZomYNMYEPJo', 'VhEbVAGQR_M', 'YC3wNH2SH9A', 'MpshWU5a7qI', 'zYsdcj0Pwzo', '_as0zT4I9X0', 'CkMuLBTHTyM', 'VaS49BxXU5o', 'eT32UFzA7E8', '9EUVRPSWJsk', 'frYiwSvylSo', 'xoBA_zhVe_c', 'OYn-quFT51M', '04SfLalN29E', 'lVg92W8u

In [32]:
video_data = []

for item in unique_ids:
    request = youtube.videos().list(
        part="snippet, statistics",
        id=item
    )
    response = request.execute()
    
    for video in response.get("items", []):
        title = video["snippet"].get("title", "")
        stats = video.get("statistics", {})
        views = int(stats.get("viewCount", 0))
        likes = int(stats.get("likeCount", 0))
        comments = int(stats.get("commentCount", 0))
        upload_time = video["snippet"].get("publishedAt", "")
        
        video_data.append({
            "title": title,
            "views": views,
            "likes": likes,
            "comments": comments,
            "upload time": upload_time
            
        })

#sort by view count
sorted_videos = sorted(video_data, key=lambda x: x['views'], reverse=True)

for item in sorted_videos:
    print(item)

{'title': 'Artikel 13 & Fridays for Future: Die Jugend demonstriert | heute-show vom 08.03.2019', 'views': 1918679, 'likes': 70458, 'comments': 5712, 'upload time': '2019-03-08T22:43:30Z'}
{'title': 'Autofans vs. Fridays for Future: Gas geben oder Welt retten? | WDR Doku', 'views': 505918, 'likes': 5417, 'comments': 5955, 'upload time': '2019-08-13T17:00:03Z'}
{'title': '#EUropeans: Fridays for future', 'views': 376917, 'likes': 401, 'comments': 0, 'upload time': '2019-10-23T10:50:11Z'}
{'title': 'Fridays For Future – SchülerInnen streiken, um die Welt zu retten', 'views': 321846, 'likes': 10308, 'comments': 3892, 'upload time': '2019-01-31T17:37:35Z'}
{'title': 'Fridays for Future: Klimaretter oder nerviger Hype? | reporter', 'views': 255967, 'likes': 3926, 'comments': 5569, 'upload time': '2019-10-02T14:30:02Z'}
{'title': 'Fridays For Future - Ein Haufen HEUCHLER?!', 'views': 176113, 'likes': 10096, 'comments': 2654, 'upload time': '2019-02-11T16:19:51Z'}
{'title': 'Our House is on F

In [33]:
import pandas as pd
#Counts how many videos were uploaded on each day
# Make sure upload_time is datetime
df["upload time"] = pd.to_datetime(df["upload time"])

# Create a year-month column
df["year_month"] = df["upload time"].dt.to_period("M")

# Count uploads per year-month
video_counts = df["year_month"].value_counts().sort_index()

print(video_counts)


year_month
2019-01     4
2019-02     3
2019-03    16
2019-04     8
2019-05    14
2019-06     5
2019-08     7
2019-09    23
2019-10    11
2019-11     8
2019-12     5
2020-01     7
2020-02     3
2020-03     2
2020-04     7
2020-05     4
2020-06     3
2020-08     3
2020-09     4
2020-10     4
2020-11     3
2020-12     3
Freq: M, Name: count, dtype: int64


C:\Users\momok\AppData\Local\Temp\ipykernel_20080\350329639.py:7: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["year_month"] = df["upload time"].dt.to_period("M")


In [34]:
from collections import Counter
import pandas as pd

keywords_list = []
for item in final_data:
    # Convert item to string just in case (comments may be lists)
    text = str(item)
    kw_single = model.extract_keywords(text, keyphrase_ngram_range=(1,1))
    kw_double = model.extract_keywords(text, keyphrase_ngram_range=(1,2))
    keywords_list.extend(kw_single + kw_double)

# Extract only keyword strings (ignore scores)
ignore_words = ["fridays", "fridays future", "future", "fridaysforfuture", "auf", "thunberg","ich", "nicht", "greta", "klima", "und"]
flattened_keywords = [kw for kw, score in keywords_list if kw.lower() not in ignore_words]


# Count frequencies
keywords_df = pd.DataFrame(Counter(flattened_keywords).items(), columns=["words", "numbers"])
frequent_words = keywords_df.loc[keywords_df['numbers'] > 1].sort_values(by='numbers', ascending=False)
print(frequent_words.head(10))




                        words  numbers
107                   climate       32
100               klimastreik       30
4              greta thunberg       19
743               klimaschutz       15
776                   twitter       12
777                 instagram       12
778  fridaysforfuture twitter       11
161            climate change       11
62                     straße       10
775           fridayforfuture        9
